# Библиотека PennyLane

PennyLane - библиотека Python для квантового машинного обучения, которую также можно использовать и для обычных квантовых вычислений. Программы, написанные на PennyLane, можно запускать используя в качестве бэкенда настоящие квантовые компьютеры от IBM Q, Xanadu, Rgetti и т.д. либо квантовые симуляторы.

Кубиты в PennyLane называются по-особому - wires (от англ. провода). Такое название, скорее всего, связано с тем, что на квантовых схемах кубиты изображаются в виде продольных линий.

Последовательность квантовых операций называется квантовой функцией. Такая функция может принимать в качестве аргументов только хэшируемые объекты. В качестве возвращаемого значения выступают величины, связанные с результатами измерения: ожидаемое значение, вероятности состояний или результаты сэмплирования. 

Квантовая функция существует не сама по себе, она запускается на определенном устройстве - симуляторе либо настоящем квантовом компьютере. Такое устройство в PennyLane называется device.

### QNode

Квантовые вычисления при использовании PennyLane раскладываются на отдельные узлы, которые называются QNode. Для их создания используются квантовые функции совместно с device.

Создавать объекты квантовых узлов можно двумя способами - явно либо с помощью декоратора qnode.

Рассмотрим первый способ - явное создание узла.

In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
dev = qml.device('default.qubit', shots=1000, wires=2, analytic=False)

In [3]:
def make_entanglement():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.probs(wires=[0, 1])

In [4]:
circuit = qml.QNode(make_entanglement, dev)

In [5]:
circuit()

tensor([0.487, 0.   , 0.   , 0.513], requires_grad=False)

Помимо прочего, обратите внимание на то, что, работая с библиотекой PennyLane, для математических операций можно использовать интерфейс Numpy, но при этом также пользоваться преимуществами автоматической дифференциации, которую обеспечивает <a href="https://github.com/HIPS/autograd">autograd</a>. 
Именно поэтому мы не импортировали Numpy обычным способом: **import numpy as np**, а сделали это таким образом:
**from pennylane import numpy as np**.

Второй способ - с помощью декоратора qnode (пропускаем импорт библиотек и создание устройства, так как вначале код тот же самый):

In [6]:
@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.probs(wires=[0, 1])

In [7]:
result = circuit()
print(result)

[0.508 0.    0.    0.492]


В данном примере мы взяли двухкубитную систему и создали запутанное состояние, а затем с помощью метода probs получили вероятности получения состояний $|00\rangle$, $|01\rangle$, $|10\rangle$, $|11\rangle$.

### Операторы

В квантовой функции можно применять операторы X, Y, Z, S, T (qml.PauliX, qml.PauliY, qml.PauliZ, qml.S, qml.T соответственно), а также операторы, в которых можно задавать угол вращения вокруг одной из осей в радианах: qml.RX, qml.RY, qml.Z. (Здесь и далее будем использовать qml как как псевдоним библиотеки pennylane).

В этой функции мы вращаем кубит под индексом 0 вокруг оси X на 90 градусов (из начального состояния $|0\rangle$) и возвращаем **ожидаемое значение** qml.PauliZ для этого кубита с помощью qml.expval. Вероятности получения состояний $|0\rangle$ и $|1\rangle$ равны, так что мы получаем ожидаемое значение, близкое к 0, что можно проверить с помощью несложных вычислений:
$$0.5 \cdot 1 + 0.5 \cdot (-1) = 0$$

In [8]:
@qml.qnode(dev)
def circuit(x):
    qml.RX(x, wires=0)
    return qml.expval(qml.PauliZ(0))

circuit(np.pi/2)

tensor(-0.032, requires_grad=True)

В следующем примере мы вращаем кубит уже вокруг оси Y, но на тот же угол 90 градусов. Ожидаемое значение в этот раз ищем для qml.PauliX, и в итоге получаем 1, что соответствует вычислениям:
$$1 \cdot 1 + 0 \cdot (-1) = 1$$

In [9]:
@qml.qnode(dev)
def circuit(x):
    qml.RY(x, wires=0)
    return qml.expval(qml.PauliX(0))

circuit(np.pi/2)

tensor(1., requires_grad=True)

При создании устройства в начале этого урока мы получили устройство, которое создает и запускает одну и ту же схему 1000 раз, каждый раз производя измерения. Поменяем этот параметр:

In [10]:
dev.shots = 5

А теперь посмотрим на результат каждого из этих пяти запусков и измерений для qml.PauliZ. Квантовая схема будет простой - применим к кубиту с индексом 1 оператор Адамара:

In [11]:
@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=1)
    return qml.sample(qml.PauliZ([1]))

circuit()

array([-1,  1,  1,  1, -1], dtype=int64)

Видно, что в результате мы получаем разные результаты: то 1 (что соответствует состоянию $|0\rangle$), то -1 (состояние $|1\rangle$).

Если вместо qml.PauliZ брать сэмплы для qml.PauliX, то ситуация поменяется, и результат все время будет один и тот же: 1, что соответствует состоянию $|+\rangle$ (вектор базиса Адамара). 

In [12]:
@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=1)
    return qml.sample(qml.PauliX([1]))

circuit()

array([1, 1, 1, 1, 1], dtype=int64)